In [36]:
import streamlit as st

import geemap, ee
import geopandas as gpd
from shapely.geometry import Polygon

st.set_page_config(layout="wide")
@st.cache_data
def ee_authenticate(token_name="EARTHENGINE_TOKEN"):
    geemap.ee_initialize(token_name=token_name)

def getNDVI(image):    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)

def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())


st.title("NDVI Map Canterbury Region")
ee_authenticate(token_name="EARTHENGINE_TOKEN")

aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME','Canterbury')).geometry()

NDVI_data = ee.ImageCollection('COPERNICUS/S2_SR').filterDate("2023-01-01","2023-01-31").filterBounds(aoi) \
    .map(getNDVI).map(addDate).median()

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
pallete = {"min":0, "max":1, 'palette':color}
# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 7)
map1.addLayer(NDVI_data.clip(aoi).select('NDVI'), pallete, "NDVI-Canterbury")

map1.addLayerControl()
map1

# map1.to_streamlit(height=700)


2023-05-30 15:06:04.064 No runtime found, using MemoryCacheStorageManager


Map(center=[-43.64309971284063, 171.44533925780135], controls=(WidgetControl(options=['position', 'transparent…

In [31]:
# [169.97634820153186, -44.7778847869712]


testPoint = ee.Feature(points.first())
#Map.centerObject(testPoint, 10)
chart = ui.Chart.image.series({
    'imageCollection': collection.select('ndvi'),
    'region': testPoint.geometry()
    }).setOptions({
      'interpolateNulls': True,
      'lineWidth': 1,
      'pointSize': 3,
      'title': 'NDVI over Time at a Single Location',
      'vAxis': '{title': 'NDVI'},
      'hAxis': '{title': 'Date', 'format': 'YYYY-MMM', 'gridlines': '{count': 12}}
    })
print(chart)


In [ ]:
ee_authenticate(token_name="EARTHENGINE_TOKEN")

aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME','Canterbury')).geometry()

coordcan = aoi.coordinates().getInfo()
# polygons = [Polygon(polygon_coords) for polygon_coords in coordcan]

In [11]:
import datetime
roi_start_time = datetime.time(14, 00)
roi_end_time = datetime.time(1, 00)

In [12]:
roi_start_time, roi_end_time

(datetime.time(14, 0), datetime.time(1, 0))

In [23]:
from dateutil.rrule import rrule, MONTHLY
from datetime import datetime

start_date = datetime.strptime("2022-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2023-12-31", "%Y-%m-%d")

months = [dt.strftime("%m-%Y") for dt in rrule(MONTHLY, dtstart=start_date, until=end_date)]
print(months)


['01-2022', '02-2022', '03-2022', '04-2022', '05-2022', '06-2022', '07-2022', '08-2022', '09-2022', '10-2022', '11-2022', '12-2022', '01-2023', '02-2023', '03-2023', '04-2023', '05-2023', '06-2023', '07-2023', '08-2023', '09-2023', '10-2023', '11-2023', '12-2023']


In [ ]:
import streamlit as st
import datetime as dt
import pandas as pd
from dateutil.relativedelta import relativedelta # to add days or years


## Range selector
cols1,_ = st.beta_columns((1,2)) # To make it narrower
format = 'MMM DD, YYYY'  # format output
start_date = dt.date(year=2021,month=1,day=1)-relativedelta(years=2)  #  I need some range in the past
end_date = dt.datetime.now().date()-relativedelta(years=2)
max_days = end_date-start_date

slider = cols1.slider('Select date', min_value=start_date, value=end_date ,max_value=end_date, format=format)
## Sanity check
st.table(pd.DataFrame([[start_date, slider, end_date]],
                columns=['start',
                        'selected',
                        'end'],
                index=['date']))

In [28]:
from datetime import datetime, timedelta
import calendar

# Given month string
month_str = "01-2023"

# Convert month string to datetime object
month_date = datetime.strptime(month_str, "%m-%Y")

# Extract year and month from the datetime object
year = month_date.year
month = month_date.month

# Create start_date and end_date based on the given month
start_date = datetime(year, month, 1).strftime("%Y-%m-%d")
last_day = calendar.monthrange(year, month)[1]
# Create the end date for the month
end_date = datetime(year, month, last_day).strftime("%Y-%m-%d")

# Print the start_date and end_date
print("start_date:", start_date)
print("end_date:", end_date)


start_date: 2023-01-01
end_date: 2023-01-31


In [26]:
import calendar
from datetime import datetime, timedelta

# Given year
year = 2023

# Iterate over each month of the year
for month in range(1, 13):
    # Get the last day of the month
    last_day = calendar.monthrange(year, month)[1]

    # Create the end date for the month
    end_date = datetime(year, month, last_day).strftime("%Y-%m-%d")

    # Print the end date
    print(f"End date of {calendar.month_name[month]}: {end_date}")


End date of January: 2023-01-31
End date of February: 2023-02-28
End date of March: 2023-03-31
End date of April: 2023-04-30
End date of May: 2023-05-31
End date of June: 2023-06-30
End date of July: 2023-07-31
End date of August: 2023-08-31
End date of September: 2023-09-30
End date of October: 2023-10-31
End date of November: 2023-11-30
End date of December: 2023-12-31


In [34]:
filteredCollection = collection.select('ndvi') \
  .filter(ee.Filter.bounds(testPoint.geometry()))

def func_xvr(image):
  stats = image.reduceRegion({
    'reducer': ee.Reducer.mean(),
    'geometry': testPoint.geometry(),
    'scale': 10,
    'maxPixels': 1e10
  })
  # reduceRegion doesn't return any output if the image doesn't intersect
  # with the point or if the image is masked out due to cloud
  # If there was no ndvi value found, we set the ndvi to a NoData value -9999
  ndvi = ee.List([stats.get('ndvi'), -9999]) \
    .reduce(ee.Reducer.firstNonNull())

  # Create a feature with None geometry and NDVI value and date as properties
  f = ee.Feature(None, {'ndvi': ndvi,
    'date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})
  return f

timeSeries = ee.FeatureCollection(filteredCollection.map(func_xvr
))

))

# Check the results
print(timeSeries.first())

# # Export to CSV
# Export.table.toDrive({
#     'collection': timeSeries,
#     'description': 'Single_Location_NDVI_time_series',
#     'folder': 'earthengine',
#     'fileNamePrefix': 'ndvi_time_series_single',
#     'fileFormat': 'CSV'
# })


TypeError: ee_object must be an ee.FeatureCollection

In [35]:
import geemap

map = geemap.Map()
map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>